<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-Scripts/blob/main/doc_summarize_chain_type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Load PDF by URL
- langchai: summarize
- The differency of 'chain_type'
- Openai:text-davinci-003 and ChatOpenai:gpt-3.5-turbo-16k

In [ ]:
%pip install openai -qU
%pip install langchain -qU
%pip install tiktoken -qU
%pip install colab-env -qU
%pip install pypdf -qU
%pip install unstructured -qU

In [ ]:
import os
import colab_env

Mounted at /content/gdrive


In [ ]:
import openai
from langchain.document_loaders import OnlinePDFLoader

In [ ]:
#online document

loader = OnlinePDFLoader('https://arxiv.org/pdf/2307.11278',)
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
print("There are %s document(s)."% len(docs))
print("There are %s characters in first document(page)." % len(docs[0].page_content))

There are 1 document(s).
There are 70725 characters in first document(page).


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

#openai completion api

In [ ]:
#alternative way to spicify 'openai_api_key'
openai.api_key=os.getenv('OPENAI_API_KEY')

In [ ]:
# playground: https://platform.openai.com/playground/p/default-summarize?mode=complete
# Openai token limit 4097
llm = OpenAI(temperature=0, model_name='text-davinci-003')
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)

stuff:
- Step1: PDF -> {prompt, doc} -> OpenAI
- Step2: OpenAI -> {doc_summary}

map_reduce:
- Step1: PDF -> [{doc_chunk(1), doc_chunk(2), ...}]
- Step2: [{prompt, doc_chunk(1)}, {prompt, doc_chunk(2)}, ...] -> OpenAI
- Step3: OpenAI -> [{chunk_summary(1)}, {chunk_summary(2)}, ...]
- Step4: [{chunk_summary(1)}, {chunk_summary(2)}, ...] -> OpenAI -> {doc_summary(the_summary_of_summary(1..n))}

refine
- Step1: PDF -> [{doc_chunk(1), doc_chunk(2), ...}]
- Step2: {prompt, doc_chunk(1)} -> OpenAI -> {chunk_summary(1)}
- Step3: {prompt, {chunk_summary(1), doc_chunk(2)}} -> OpanAI -> {chunk_summary(1_2)}
- ...
- StepN: {doc_summary(chunk_summary(1_n))}

map_rerank
- Step1: PDF -> [{doc_chunk(1), doc_chunk(2), ...}]
- Step2: {prompt, doc_chunk(1)} -> OpenAI -> {chunk_summary(1), rank_score(1)}
- ...
- StepN: {doc_summary(chunk_summary(k), highest_rank_score(k))}

In [ ]:
chain.run(input_documents=split_docs)

gpt-3.5-turbo-16k

In [ ]:
 content = ""
 for s in split_docs[:70]:
    content += s.page_content + "\n"

refer to: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
- https://platform.openai.com/tokenizer

In [51]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-16k')
encoding_davinci = tiktoken.encoding_for_model('text-davinci-003')

In [52]:
print(len(content))
print(len(encoding.encode(content)))
print(len(encoding_davinci.encode(content)))

53069
13847
14040


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

# llm = OpenAI(temperature=0, model_name='text-davinci-003')
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k', verbose=False)
chain = load_summarize_chain(llm, chain_type="stuff")

In [ ]:
chain.run(input_documents=split_docs[:70])

'The paper proposes a novel approach called Generator-Retriever-Generator (GRG) for open-domain question answering. The approach combines document retrieval techniques with large language models to generate informative and contextually relevant answers. The GRG approach outperforms existing baseline models on benchmark datasets, demonstrating its effectiveness in improving question answering accuracy. The paper also includes an ablation study and provides detailed implementation details and experimental results.'